In [4]:
import pandas as pd
import numpy as np
import os
from glob import glob
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import spearmanr, pearsonr

In [5]:
DATA_DIR = './'

In [6]:
def calc_MODZ(data):
    """calculates MODZ weights based on the original CMAP/L1000 study
    use only lm genes for MODZ calculation!"""
    if data.shape[1]==1: #1 column
        weights = np.array([1.0])
    elif data.shape[1]==2: # 2 columns
        weights = np.array([[0.5], [0.5]])
    else:
        CM = spearmanr(data)[0] # correlations (r) array with shape col x col
        fil = CM<0
        CM[fil] = 0.01 # correlation less than 0 = 0.01
        weights = np.sum(CM, 1)-1 # sum by rows minus the correlation woth itself (r=1)
        weights = weights / np.sum(weights)
        weights = weights.reshape((-1, 1)) # reshape to column,each value in separate array [[1][2][3]]
    return weights

In [7]:
def select_treatment_type(treatment_type):
    sig_info_selected = sig_info[sig_info['pert_type']==treatment_type]
    selected_cols = ['sig_id', 'pert_type', 'pert_id',  'cmap_name', 'cell_iname', 'pert_itime']
    sig_info_selected = sig_info_selected[selected_cols]
    sig_info_selected = sig_info_selected.set_index('sig_id', drop = True)
    return sig_info_selected
    

In [8]:
def calculate_consesus_signatures(treatment_type):
    siginf = select_treatment_type(treatment_type)
    
    cmap_names = list(set(siginf.cmap_name.dropna()))
    signatures_lm = pd.DataFrame(index=genes_lm.index, columns=cmap_names)    
    lname = glob(DATA_DIR+'level5_beta_'+treatment_type+'*.gctx')[0]

    for i in range(len(cmap_names)):
        cname = cmap_names[i]

        if i % 100 == 0:
            print(i, end = ',', flush=True)

        sample_ids = siginf[siginf['cmap_name']==cname].index
        gex_lm = parse(lname, cid=sample_ids,rid=genes_lm.index).data_df.loc[genes_lm.index]

        weights = calc_MODZ(gex_lm)
        gex_lm = pd.DataFrame(np.dot(gex_lm, weights), index=gex_lm.index, columns=[cname])

        signatures_lm[cname] = gex_lm[cname]
    signatures_lm.to_csv(DATA_DIR+'signatures_lm_'+treatment_type.split('_')[1]+'.csv')
    print('done')

In [217]:
def map_gene_names(genes_lm, treatment_type):
    genes = genes_lm.copy()
    genes.index = genes.index.astype('int')
    
    signatures = pd.read_csv(DATA_DIR+'signatures_lm_'+treatment_type.split('_')[1]+'.csv', index_col =0)
    signatures.index = signatures.index.map(genes.to_dict())
    signatures.to_csv(DATA_DIR+'signatures_lm_'+treatment_type.split('_')[1]+'_genenames.csv')


In [9]:
gene_info = pd.read_table(DATA_DIR+'geneinfo_beta.txt')
sig_info = pd.read_table(DATA_DIR+'siginfo_beta.txt', low_memory=False)

In [10]:
# every case we have landmark genes
fil = gene_info['feature_space']=='landmark'
genes_lm = gene_info[fil]
genes_lm = genes_lm.set_index('gene_id', drop = True)['gene_symbol']
genes_lm.index = genes_lm.index.astype(str)

In [45]:
# calculate_consesus_signatures('trt_oe')

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,done


In [46]:
# calculate_consesus_signatures('trt_sh') - use consesus generated by lincs

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,done


In [ ]:
# calculate_consesus_signatures('trt_xpr')

In [1]:
# calculate_consesus_signatures('trt_cp') - takes time - only desired kinase and phosphatase signatures are needed.

# Sh cell signatures 96 h 

# Cell

In [115]:
len(cmap_names)

4345

In [119]:

treatment_type = 'trt_sh.cgs'
siginf = select_treatment_type('trt_sh.cgs')
cmap_names = list(set(siginf.cmap_name.dropna()))
signatures_lm = pd.DataFrame(index=genes_lm.index, columns=cmap_names)    
lname = glob(DATA_DIR+'level5_beta_'+treatment_type.split('.')[0]+'*.gctx')[0]
cells = siginf.cell_iname.unique()
signatures_cell_lm = {}

for cell in cells:
    print(cell, end = ', ')
    signatures_lm = {}
    for i in range(len(cmap_names)):
        cname = cmap_names[i]
        if i%1000 == 0:
            print(i, end = ',', flush=True)
        sample_ids = siginf[(siginf['cmap_name']==cname) & (siginf['cell_iname']==cell) & (siginf['pert_itime']=='96 h')].index
        gex_lm = parse(lname, cid=sample_ids,rid=genes_lm.index).data_df.loc[genes_lm.index]
        
        if gex_lm.shape[1] > 0:
            assert(gex_lm.shape[1] == 1)
            signatures_lm[cname] = gex_lm
    print()
    if len(signatures_lm) >0:
        signatures_cell_lm[cell] = signatures_lm
        
print('done')

A375, 0,1000,2000,3000,4000,
MCF7, 0,1000,2000,3000,4000,
HT29, 0,1000,2000,3000,4000,
PC3, 0,1000,2000,3000,4000,
HA1E, 0,1000,2000,3000,4000,
HEPG2, 0,1000,2000,3000,4000,
VCAP, 0,1000,2000,3000,4000,
HCC515, 0,1000,2000,3000,4000,
A549, 0,1000,2000,3000,4000,
ASC, 0,1000,2000,3000,4000,
SKL, 0,1000,2000,3000,4000,
HEKTE, 0,1000,2000,3000,4000,
NPC, 0,1000,2000,3000,4000,
SW480, 0,1000,2000,3000,4000,
SHSY5Y, 0,1000,2000,3000,4000,
HEK293T, 0,1000,2000,3000,4000,
NCIH716, 0,1000,2000,3000,4000,
done


In [161]:
for cell in signatures_cell_lm:
    print(cell, end = ', ')
    idx = signatures_cell_lm[cell][list(signatures_cell_lm[cell].keys())[0]].index
    df = pd.DataFrame(columns = list(signatures_cell_lm[cell].keys()), index = idx)
    
    for i in signatures_cell_lm[cell]:
        df[i] = signatures_cell_lm[cell][i]
    
    df.to_csv(f'./cell_signatures/signatures_lm_sh_{cell}.csv')
print('done')

A375, MCF7, HT29, PC3, HA1E, HEPG2, VCAP, HCC515, A549, ASC, SKL, HEKTE, NPC, SW480, SHSY5Y, 

IndexError: list index out of range

# Sh signatures

#### MODZ consensus - all cell lines and all timepoints

In [211]:
treatment_type = 'trt_sh.cgs'
siginf = select_treatment_type('trt_sh.cgs')
cmap_names = list(set(siginf.cmap_name.dropna()))
signatures_lm = pd.DataFrame(index=genes_lm.index, columns=cmap_names)    
lname = glob(DATA_DIR+'level5_beta_'+treatment_type.split('.')[0]+'*.gctx')[0]

signatures_lm = {}
for i in range(len(cmap_names)):
    cname = cmap_names[i]
    
    if i%100 == 0:
        print(i, end = ',', flush=True)
        
    sample_ids = siginf[(siginf['cmap_name']==cname)].index        
    gex_lm = parse(lname, cid=sample_ids,rid=genes_lm.index).data_df.loc[genes_lm.index]

    weights = calc_MODZ(gex_lm)
    gex_lm = pd.DataFrame(np.dot(gex_lm, weights), index=gex_lm.index, columns=[cname])

    signatures_lm[cname] = gex_lm[cname]
signatures_lm.to_csv(DATA_DIR+'signatures_lm_'+treatment_type.split('_')[1]+'.csv')
print('done')

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,

AttributeError: 'dict' object has no attribute 'to_csv'

In [215]:
signatures_lm_df = pd.DataFrame(signatures_lm, columns = signatures_lm.keys(), index = signatures_lm['HDAC2'].index)
# signatures_lm_df.to_csv(DATA_DIR+'signatures_lm_'+treatment_type.split('_')[1]+'.csv')


In [219]:
map_gene_names(genes_lm, 'trt_sh.cgs')

## map gene names

In [60]:
map_gene_names(genes_lm, 'trt_xpr')

# Calculate signatures only for cps with phosphatase or kinase targets

In [51]:
treatment_type = 'trt_cp'
siginf = select_treatment_type(treatment_type)

In [52]:
cpd_info = pd.read_table(DATA_DIR+'compoundinfo_beta.txt')

In [53]:
metadata = pd.read_csv('sasaki_data.csv', index_col = 0)
metadata = metadata.set_index('GeneSymbol', drop=True)

In [54]:
kinases = list(metadata[metadata['Function'] == 'kinase'].index)
phosphatases = list(metadata[metadata['Function'] == 'phosphatase'].index)

In [55]:
cpd_info = cpd_info[cpd_info.target.isin(kinases+phosphatases)]

In [137]:
siginf = siginf[siginf.pert_id.isin(cpd_info.pert_id)]

In [138]:
cmap_names = list(set(siginf.cmap_name.dropna()))
signatures_lm = pd.DataFrame(index=genes_lm.index, columns=cmap_names)    
lname = glob(DATA_DIR+'level5_beta_'+treatment_type+'*.gctx')[0]

In [143]:
for i in range(len(cmap_names)):
        cname = cmap_names[i]

#         if i % 100 == 0:
#             print(i, end = ',', flush=True)

        sample_ids = siginf[siginf['cmap_name']==cname].index
        gex_lm = parse(lname, cid=sample_ids,rid=genes_lm.index).data_df.loc[genes_lm.index]

        weights = calc_MODZ(gex_lm)
        gex_lm = pd.DataFrame(np.dot(gex_lm, weights), index=gex_lm.index, columns=[cname])

        signatures_lm[cname] = gex_lm[cname]
signatures_lm.to_csv(DATA_DIR+'signatures_lm_'+treatment_type.split('_')[1]+'_KandP.csv')

0,

In [146]:
signatures = pd.read_csv(DATA_DIR+'signatures_lm_cp_KandP.csv', index_col =0)
genes = genes_lm.copy()
genes.index = genes.index.astype('int')
signatures.index = signatures.index.map(genes.to_dict())
signatures.to_csv(DATA_DIR+'signatures_lm_cp_KandP_genenames.csv')

In [147]:
signatures = pd.read_csv(DATA_DIR+'signatures_lm_cp_KandP_genenames.csv', index_col =0)